<a href="https://colab.research.google.com/github/Adhambarakat/supreme-chainsaw/blob/main/regressionXiotloadmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Dependencies

In [9]:
!pip install influxdb-client
!pip install flightsql-dbapi
!pip install pandas
!pip3 install paho-mqtt

Importing Libraries

In [10]:
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
import time
import paho.mqtt.client as paho
from paho import mqtt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.engine.training import optimizer
import matplotlib.pyplot as plt
import keras.utils as to_categorica
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

Allocating Data

In [16]:
#allocating data
data = pd.read_excel('/content/a7a6.xlsx')
input_dim=1
PPG = data['PPG']  #ptt training
SBP = data['SBP']  #systolic
DBP = data['DBP']  #diastolic
EXP = np.column_stack((SBP,DBP))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(PPG,EXP, random_state=42, train_size=.8)

Defining the Model

In [ ]:
#defining the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=16, activation='relu', input_shape=[input_dim]),
    tf.keras.layers.Dense(units=2, input_shape=[input_dim])
])

Compiling the Model

In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

Training the Model

In [ ]:
model.fit(abs(x_train), y_train, epochs=10, batch_size=100)
model.save('model.h5')

Epoch 1/10
6292/6292 [==============================] - 15s 2ms/step - loss: 2046.2035 - mae: 30.3627 - val_loss: 340.3427 - val_mae: 13.4921
Epoch 2/10
6292/6292 [==============================] - 14s 2ms/step - loss: 125.3334 - mae: 8.4710 - val_loss: 291.8344 - val_mae: 12.6877
Epoch 3/10
6292/6292 [==============================] - 15s 2ms/step - loss: 115.8575 - mae: 8.0166 - val_loss: 290.7965 - val_mae: 12.7028
Epoch 4/10
6292/6292 [==============================] - 15s 2ms/step - loss: 115.8615 - mae: 8.0173 - val_loss: 291.6068 - val_mae: 12.7100
Epoch 5/10
6292/6292 [==============================] - 14s 2ms/step - loss: 115.8610 - mae: 8.0171 - val_loss: 290.1551 - val_mae: 12.6919
Epoch 6/10
6292/6292 [==============================] - 13s 2ms/step - loss: 115.8591 - mae: 8.0171 - val_loss: 288.6693 - val_mae: 12.6674
Epoch 7/10
6292/6292 [==============================] - 13s 2ms/step - loss: 115.8651 - mae: 8.0169 - val_loss: 291.1793 - val_mae: 12.7312
Epoch 8/10
6292/62

In [13]:
model=tf.keras.models.load_model('model.h5')

In [ ]:
loss, mae = model.evaluate(x_test, y_test)

Connecting to Influxdb

In [14]:
# !export INFLUXDB_TOKEN=-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg==

#token = os.environ.get("INFLUXDB_TOKEN")
token = "-g6iC2cK-mwZpYCPaiKdDOyyX5P1vkncIDSigkxs_ImaQJUDUm1-87FX4peKEmW4F2DC9CMSqZKVVTLNvXERSg=="
org = "test"
url = "https://us-east-1-1.aws.cloud2.influxdata.com"

write_client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

x = 123
print (x)

123


Receiving and Sending Data

In [17]:
# setting callbacks for different events to see if it works, print the message etc.
def on_connect(client, userdata, flags, rc, properties=None):
    print("CONNACK received with code %s." % rc)

# with this callback you can see if your publish was successful
def on_publish(client, userdata, mid, properties=None):
    print("mid: " + str(mid))

# print which topic was subscribed to
def on_subscribe(client, userdata, mid, granted_qos, properties=None):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

# print message, useful for checking if it was successful
def on_message(client, userdata, msg):
    #print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    #print(int(msg.payload))
    if msg.topic == "avgbpm" :
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point1": {
          "location": "Hospital",
          "dataa": "avghr",
          "valuee": int(msg.payload),
        },
      }
      for key in data:
        point = (
          Point("avghr")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(msg.payload)
    elif msg.topic == "ppg" :
      x_new=[[float(int(msg.payload)/100)]]
      y_new = model.predict(np.array([[x_new]]))
      a = y_new[0]
      b = a[0]
      c = b[0]
      sbp = int(c[0])
      dbp = int(c[1])
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point2": {
          "location": "Hospital",
          "dataa": "sbp",
          "valuee": sbp,
        },
      }
      for key in data:
        point = (
          Point("sp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point3": {
          "location": "Hospital",
          "dataa": "dbp",
          "valuee": dbp+20,
        },
      }
      for key in data:
        point = (
          Point("dp")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(sbp)
      print(dbp+20)
    else :
      bucket="colabtest"
      # Define the write api
      write_api = write_client.write_api(write_options=SYNCHRONOUS)
      data = {
        "point4": {
          "location": "Hospital",
          "dataa": "acchr",
          "valuee": int(msg.payload),
        },
      }
      for key in data:
        point = (
          Point("acchr")
          .tag("location", data[key]["location"])
          .field(data[key]["dataa"], data[key]["valuee"])
        )
      write_api.write(bucket=bucket, org=org, record=point)
      time.sleep(1) # separate points by 1 second
      print("Complete. Return to the InfluxDB UI.")
      print(msg.payload)


    """
    bucket="colabtest"
    # Define the write api
    write_api = write_client.write_api(write_options=SYNCHRONOUS)
    data = {
      "point4": {
        "location": "Hospital",
        "dataa": "hr",
        "valuee": int(msg.payload),
      },
    }
    for key in data:
      point = (
        Point("bps")
        .tag("location", data[key]["location"])
        .field(data[key]["dataa"], data[key]["valuee"])
      )
    write_api.write(bucket=bucket, org=org, record=point)
    time.sleep(1) # separate points by 1 second
    print("Complete. Return to the InfluxDB UI.")
    """



# using MQTT version 5 here, for 3.1.1: MQTTv311, 3.1: MQTTv31
# userdata is user defined data of any type, updated by user_data_set()
# client_id is the given name of the client
client = paho.Client(client_id="", userdata=None, protocol=paho.MQTTv5)
client.on_connect = on_connect

# enable TLS for secure connection
client.tls_set(tls_version=mqtt.client.ssl.PROTOCOL_TLS)
# set username and password
client.username_pw_set("ahmed", "Ahmed1603")
# connect to HiveMQ Cloud on port 8883 (default for MQTT)
client.connect("7ea331cfad864375a4a189652c9375e0.s1.eu.hivemq.cloud", 8883)

# setting callbacks, use separate functions like above for better visibility
client.on_subscribe = on_subscribe
client.on_message = on_message
client.on_publish = on_publish

# subscribe to all topics of encyclopedia by using the wildcard "#"
client.subscribe("bpm", qos=1)
client.subscribe("ppg", qos=1)
client.subscribe("avgbpm", qos=1)

# a single publish, this can also be done in loops, etc.
# client.publish("encyclopedia/temperature", payload="hot", qos=1)

# loop_forever for simplicity, here you need to stop the loop manually
# you can also use loop_start and loop_stop
client.loop_forever()

CONNACK received with code Success.
Subscribed: 1 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7cf7c1cc66e0>]
Subscribed: 2 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7cf7c1cc6710>]
Subscribed: 3 [<paho.mqtt.reasoncodes.ReasonCodes object at 0x7cf7c1cc58a0>]
Complete. Return to the InfluxDB UI.
b'80'
1/1 [==============================] - 0s 41ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
122
71
Complete. Return to the InfluxDB UI.
b'78'
1/1 [==============================] - 0s 25ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
125
69
Complete. Return to the InfluxDB UI.
b'87'
1/1 [==============================] - 0s 27ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
127
68
Complete. Return to the InfluxDB UI.
b'80'
1/1 [==============================] - 0s 23ms/step
Complete. Return to the InfluxDB UI.
Complete. Return to the InfluxDB UI.
123
70
Complete. Return to the Influ

KeyboardInterrupt: ignored